In [1]:
import win32api
import win32con
import win32gui
import time
import torch
from torchvision import transforms
from kanji_detection_model import kanji_detector
import json


from PIL import ImageGrab, Image

#for testing purposes
import pathlib
from os import listdir
from os.path import isfile, join
import random
import math

In [2]:
num2kanji_dict = {}
with open('num2kanji_dict.json', 'r') as f:
  num2kanji_dict = json.load(f)


device = "cuda" if torch.cuda.is_available() else "cpu"
#detectionModel = torch.load('./Models/kanji_model_96_1.pt', map_location=torch.device('cpu'))
detectionModel = kanji_detector().to(device='cpu')
#detectionModel.load_state_dict(torch.load('./Models/kanji_model_98_1.pth'))
#detectionModel.load_state_dict(torch.load('./Models/kanji_model_98_1_pertub.pth'))
detectionModel.load_state_dict(torch.load('./Models/kanji_model_top5_99_pertub.pth'))

<All keys matched successfully>

In [3]:
def sortAccumulator(item):
    return item[1]

def identifySymbol(top_k: int = 5):
    
    capture_sizes = [30,40,50]
    half_capture_sizes = [capture_size//2 for capture_size in capture_sizes]
    
    convert_tensor = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(64),
        transforms.ToTensor()
    ])
    
    accumulator = {}
    
    with torch.no_grad():
        activeModel = detectionModel.to(device=device)
        activeModel.eval()
        
        maxKanji = u"魔"
        maxValue = -9999999999.0
        
        for i in range(len(capture_sizes)):
            
            half_capture_size = half_capture_sizes[i]
            
            posClick = win32gui.GetCursorPos()
            rect = (posClick[0]-half_capture_size, 
                    posClick[1]-half_capture_size,
                    posClick[0]+half_capture_size,
                    posClick[1]+half_capture_size)
            image = ImageGrab.grab(bbox=rect)
            
            tensor_image = convert_tensor(image)
            
            results = activeModel(tensor_image.unsqueeze(0).to(device=device))
            results = torch.nn.functional.softmax(results, dim=-1)
            
            
            #top_values, most_likely_indices = results.squeeze().topk(k=top_k,dim=0)
            top_values, most_likely_indices = results.squeeze().topk(k=results.size(-1),dim=0)
            selections = (most_likely_indices).tolist()
            selections_probs = top_values.tolist()
            
            kanjis = [num2kanji_dict[str(k)] for k in selections]
            #print(kanjis)
            
            for i in range(len(selections)):
                k = num2kanji_dict[str(selections[i])]
                v = selections_probs[i]
                
                if maxValue < v:
                    maxKanji = k
                    maxValue = v
                
                accumulator[k] = accumulator.get(k,0) + v
            
        #print("")
        acc_list = list(accumulator.items())
        acc_list.sort(key=sortAccumulator,reverse=True)
        clean_list = [item[0] for item in acc_list]
        print(maxKanji)
        print(clean_list[:20])
        
        
        print("_________________________________________")
        
        del activeModel
        del tensor_image
    

In [ ]:
def isPressed(key) -> bool:
    return (win32api.GetKeyState(key) & ~1) != 0 # Left button down = 0 or 1. Button up = -127 or -128

capture_size = 40
half_capture_size = capture_size//2

state_left = isPressed(win32con.VK_LBUTTON)
state_ctrl = isPressed(win32con.VK_CONTROL)

while True:
    a = isPressed(win32con.VK_LBUTTON)
    state_ctrl = isPressed(win32con.VK_CONTROL)
    
    if a != state_left: # Button state changed
        state_left = a
            
        if a and state_ctrl:
            isHolding = True
            """
            posClick = win32gui.GetCursorPos()
            rect = (posClick[0]-half_capture_size, 
                    posClick[1]-half_capture_size,
                    posClick[0]+half_capture_size,
                    posClick[1]+half_capture_size)
            pic = ImageGrab.grab(bbox=rect)
            identifySymbol(pic, top_k=10)
            """
            identifySymbol(top_k=5)
            #print('Left Button Pressed')
                

誰
['誰', '渓', '離', '英', '舌', '能', '髄', '酷', '寒', '雑', '待', '空', '渉', '茂', '食', '笠', '為', '浅', '弟', '楽']
_________________________________________
探
['探', '辞', '抹', '採', '珠', '弟', '挨', '控', '栽', '粋', '恭', '報', '拝', '界', '閑', '兼', '搾', '保', '様', '弔']
_________________________________________
薄
['薄', '黄', '事', '専', '連', '準', '要', '重', '壌', '簿', '菓', '費', '凄', '碁', '異', '華', '軍', '基', '革', '逮']
_________________________________________
史
['送', '矢', '史', '沃', '大', '天', '女', '丈', '妖', '太', '失', '犬', '夫', '父', '欠', '央', '友', '英', '扶', '与']
_________________________________________
入
['入', '込', '久', '人', '大', '木', '又', '太', '火', '欠', '刀', '汰', '双', '文', '本', '玄', '乃', '水', '奴', '衣']
_________________________________________
廃
['廃', '魔', '庭', '療', '歴', '険', '磨', '除', '嬢', '陸', '族', '賄', '醸', '駐', '降', '逸', '短', '商', '慰', '旗']
_________________________________________
世
['世', '抽', '抵', '伴', '笹', '油', '担', '推', '追', '拙', '池', '把', '曲', '他', '描', '稚', '牲', '進', '弾', '准']
___________________________

In [5]:
trainingPath = pathlib.Path().resolve() / "Training_set"
picturesNames = [f for f in listdir(trainingPath) if isfile(join(trainingPath, f))]

dictNames = {name:{'number':int(name.split('_')[0]) , 'symbol':name.split('_')[1]} for name in picturesNames}
tempDictNames = dictNames.copy()

image_size=64
nb_symbols=2199
#print(dictNames[picturesNames[0]])

def selectBatches(dictNames, tempDictNames, batch_size) -> list:
    batch=[]
    while len(batch) < batch_size:
        n_to_find = batch_size-len(batch)
        if len(tempDictNames) >= n_to_find:
            sample = random.sample(list(tempDictNames.items()), n_to_find)
            batch.extend(sample)
            for item in sample:
                del tempDictNames[item[0]]
        else:
            batch.extend(tempDictNames.items())
            tempDictNames = dictNames.copy()

    return batch

def getAnswerIndices(batchList) -> torch.FloatTensor:
    
    correctAnswer = torch.zeros((len(batchList),nb_symbols)).float()
    correctAnswerIndices = torch.zeros(len(batchList)).long()
    
    for i in range(len(batchList)):
        indexCorrect = batchList[i][1]['number']-1
        correctAnswer[i][indexCorrect] = 1
        correctAnswerIndices[i] = indexCorrect
    
    
    return correctAnswer, correctAnswerIndices

def countCorrect(answer: torch.FloatTensor, correctAnswer: torch.FloatTensor):
    
    
    _,indicesAnswer = torch.max(answer, dim=1)
    _,indicesCorrect = torch.max(correctAnswer, dim=1)
    
    #print(indicesAnswer)
    #print(indicesCorrect)
    numCorrect = (indicesAnswer == indicesCorrect).long().sum()
    
    return numCorrect.item()
    
def getPictures(batchList) -> torch.FloatTensor:
    
    images = torch.zeros((len(batchList), 1, image_size, image_size)).float()
    convert_tensor = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])
    
    for i in range(len(batchList)):
        item = batchList[i]
        filename = item[0]
        img = Image.open(trainingPath / filename)
        images[i,:,:,:] = convert_tensor(img)
        
    return images

In [6]:
def eval(model, n_epoch, batch_size):
    n_batches = 1
    
    with torch.no_grad():

        model.eval()
        loss_f = torch.nn.CrossEntropyLoss()
        best_percent = 0
        for epoch in range(n_epoch):
            #print("Epoch " + str(epoch+1) + " is running")
            n_correct = 0
            n_total = n_batches*batch_size
            for i in range(n_batches):

                batch = selectBatches(dictNames, tempDictNames, batch_size)
                images = getPictures(batch)
                correct_answer, correct_answer_indices = getAnswerIndices(batch)

                answer = model(images.to(device=device))
                loss = loss_f(answer,correct_answer_indices.to(device=device)).cpu()

                print(answer)
                n_correct += countCorrect(answer, correct_answer.to(device=device))

            adjust = 10000
            percent = math.floor(adjust*100*n_correct/n_total)/adjust
            best_percent = percent if percent > best_percent else best_percent
            print("Epoch " + str(epoch+1) +" evaluation accuracy : " + str(percent) + "%\n")
        
    return best_percent

In [7]:
eval(detectionModel,1,100)

tensor([[-3.7491, -2.7350, -3.2033,  ..., -3.8105, -3.3552, -2.9861],
        [-2.9835, -2.7410, -3.6033,  ..., -3.2384, -2.8220, -2.2905],
        [-3.8220, -2.8390, -4.6016,  ..., -3.8309, -3.0583, -2.7301],
        ...,
        [-3.5030, -2.5716, -2.9183,  ..., -3.5637, -3.3498, -2.9802],
        [-3.4549, -2.2524, -2.9277,  ..., -3.3852, -3.1757, -3.4982],
        [-3.6615, -3.1105, -2.6145,  ..., -3.4711, -3.3693, -2.9698]],
       device='cuda:0')
Epoch 1 evaluation accuracy : 7.0%



7.0

In [3]:
width = win32api.GetSystemMetrics(0)
height = win32api.GetSystemMetrics(1)
midWidth = int((width + 1) / 2)
midHeight = int((height + 1) / 2)


"""
dc = win32gui.GetDC(0)
dcObj = win32ui.CreateDCFromHandle(dc)
hwnd = win32gui.WindowFromPoint((0,0))
"""
rect = (0, 0, width, height)
window = (0, 0, width, height)


app = wx.App(False)
s = wx.ScreenDC()
s.Pen = wx.Pen(wx.Colour(0,0,0,255),style=wx.TRANSPARENT)
s.StartDrawingOnTop()



state_left = isPressed(win32con.VK_LBUTTON)
state_ctrl = isPressed(win32con.VK_CONTROL)
print(state_left)
print(state_ctrl)

isHolding = False

posStart = win32gui.GetCursorPos()
posEnd = win32gui.GetCursorPos()

brush = CreateBrush()
brush.CreateSolidBrush(win32api.GetSysColor(255))



False
False


In [3]:
class DrawPanel(wx.Panel):
    
    def __init__(self, *args, **kw):
        super().__init__(*args, **kw)
        
        self.Bind(wx.EVT_PAINT, self.on_paint)
        self.Bind(wx.EVT_MOTION, self.on_mouse_motion)
        
    def on_mouse_motion(self, event):
        self.Parent.Refresh()
        
    def on_paint(self, event):
        self.dc = wx.PaintDC(self)
        #self.dc.DrawRectangle(rect)
        self.dc.DrawRectangle(0,0,100,100)

In [4]:

while True:
    a = isPressed(win32con.VK_LBUTTON)
    state_ctrl = state_ctrl if state_left else isPressed(win32con.VK_CONTROL)
    
    if (a != state_left): # Button state changed
        state_left = a
        if state_ctrl:
            
            if a:
                isHolding = True
                posStart = win32gui.GetCursorPos()
                posEnd = win32gui.GetCursorPos()
                #hwnd = win32gui.WindowFromPoint((0,0))
                print('Left Button + Ctrl Pressed')
                
                
            else:
                isHolding = False
                posEnd = win32gui.GetCursorPos()
                print('Left Button + Ctrl Released')
                
                s.Clear()
                
                #win32gui.InvalidateRect(hwnd, window, False)
                #win32api.SetCursorPos((midWidth, midHeight))
        
    if isHolding:
        posEnd = win32gui.GetCursorPos()
        """
        win32gui.RedrawWindow(hwnd, None, None, win32con.RDW_INVALIDATE | win32con.RDW_ERASE)
        win32gui.UpdateWindow(hwnd)
        rect = (posStart[0], posStart[1], posEnd[0], posEnd[1])
        dcObj.FrameRect(rect,brush)
        """
        
        
        
        rect = (posStart[0], posStart[1], posEnd[0]-posStart[0], posEnd[1]-posStart[1])
        s.DrawRectangle(rect)
        s.Refresh()
        #app.Update()
        
        
        rect = (posStart[0], posStart[1], posEnd[0]-posStart[0], posEnd[1]-posStart[1])
    
    time.sleep(0.01)
s.EndDrawingOnTop()

Left Button + Ctrl Pressed
Left Button + Ctrl Released
Left Button + Ctrl Pressed
Left Button + Ctrl Released
Left Button + Ctrl Pressed
Left Button + Ctrl Released


KeyboardInterrupt: 